<a href="https://colab.research.google.com/github/james00000007/Google-Colaboratory/blob/main/Torrent%20to%20Gdrive%E2%9C%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown  ← Current Python Version
!python -V

In [ ]:
#@markdown <h3> ⬅️ Prevent Google Colab from disconnecting | AUTO RECONNECT
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

In [ ]:
#@markdown <h3>⬅ Mount Google Drive</h3>
import os, uuid; import ipywidgets as widgets
from IPython.display import clear_output
from google.colab import drive, output
 
class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)
    
def DemountForm():
  clear_output(wait=True)
  display(MakeLabel("Demount Confirm?", ""), MakeButton("Confirm", Demount, "danger"), MakeButton("Cancel", ShowGD, "warning"))
    
def Demount():
  clear_output(wait=True)
  get_ipython().system_raw("pkill -9 -e -f drive")
  !rm -rf /root/.config/Google/DriveFS/
  !echo "If you want to mount Google Drive run this cell again."
  
def ShowGD():
  try:
    RefreshPath()
  except:
    clear_output(wait=True)
  display(MakeLabel("Control Panel", ""), MakeButton("Demount", DemountForm, "danger"))
 
while not os.path.exists("/content/drive"):
  try:
    drive.mount("/content/drive")
    clear_output(wait=True)
  except:
    clear_output(wait=True)
    
ShowGD()

In [ ]:
#@markdown <h3>⬅ To Mount Another Gdrive!</h3>
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!sudo mkdir /content/drive1
!google-drive-ocamlfuse /content/drive1

## <img src='https://colab.research.google.com/img/colab_favicon_256px.png' height="30" alt="colab" />  __Torrent to Gdrive__



In [ ]:
#@title
#@markdown <h3>⬅️ Click Here to START server</h3>

!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent

!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

from IPython.display import HTML, clear_output

clear_output()

print("Server Started Successfully")



In [ ]:
#@title
#@markdown <h3>⬅️ Click Here to </h3>
#@markdown <h3>Upload torrent file</h3>
#@markdown <h5>You can run this cell to add more files as many times as you want</h5><br>



from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/Torrent",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

In [ ]:
#@title
#@markdown <h3>⬅️ Click Here to </h3>
#@markdown <h3>Add From Magnet Link</h3>
#@markdown <h5>You can run this cell to add more files as many times as you want</h5><br>


params = {"save_path": "/content/drive/"}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )


In [ ]:
#@title
#@markdown <h3>⬅️ Click Here to Start Download</h3>
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)
